In [1]:
import json
from tqdm.auto import tqdm
import random
random.seed(42)
with open('../data/chronicle/chronicle_data.json') as f:
    data = json.load(f)

In [2]:
random.shuffle(data)

In [3]:
import os
os.environ['HF_HOME'] = './'

In [4]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="JetBrains-Research/cmg-race-with-history", device=0)

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\TU Delft\thesis\LLM_CMG\llm4commit\models\hub\models--JetBrains-Research--cmg-race-with-history. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/511k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

In [5]:
diffs = []
for commit in data[:100]:
    diff = commit['diff']
    diffs.append(diff)

generated_commit_messages = []
for diff in tqdm(diffs, desc='Generating commit messages'):
    prompt = f"""
    The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly.
    {diff}
    According to the diff, the commit message should be:
    """
    generated_commit_messages.append(pipe(prompt)[0]['generated_text'])

Generating commit messages:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 1100, in emit
    msg = self.format(record)
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 943, in format
    return fmt.format(record)
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 678, in format
    record.message = record.getMessage()
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.

In [6]:
for item, msg in zip(data[:100], generated_commit_messages):
    item['race_history'] = msg

output_file = '../data/chronicle/chronicle_race_history.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(data[:100], f, ensure_ascii=False, indent=4)